## Required Imports

In [1]:
import h5py
import matplotlib.pyplot as plt
import json
import numpy as np
import plotly.graph_objects as go
import lmfit as lm

from common import save_to_igor_itx, single_wave_masks, convert_to_4_setpoint_AW, calculate_fit, i_sense, entropy_weak
from squarewave import process_per_row_parts, SquareProcessedData, process_avg_parts, get_data_part

## Data -- Bad heating potential (TODO: Currently this is just a copy of Fig 3)


In [8]:
with h5py.File('data/dat1146.h5', 'r') as f:
    data = f['cscurrent_2d'][:]
    x = f['x_array'][:]
    sweeplogs = f['metadata'].attrs['sweep_logs']
    aw = f['fdAW_0'][:]
sweeplogs = json.loads(sweeplogs)
awg_info = sweeplogs['FastDAC 1']['AWG']
num_pts = awg_info['waveLen']
num_steps = awg_info['numSteps']
aw = convert_to_4_setpoint_AW(aw)

In [9]:
processed = process_per_row_parts(x=x, data=data, arbitrary_wave=aw, setpoint_start=13, num_steps=num_steps,num_cycles=1)
cold_data = get_data_part(processed.cycled, 'cold')

In [10]:
params = lm.Parameters()
params.add_many(
    ('mid', 0, True, None, None, None, None),
    ('theta', 5, True, 0.01, None, None, None),
    ('amp', 1, True, 0, None, None, None),
    ('lin', 0.001, True, 0, None, None, None),
    ('const', 5, True, None, None, None, None)
)

fits = [calculate_fit(x, d, params, func=i_sense) for d in data]

centers = [fit.params['mid'].value for fit in fits]

In [11]:
processed = process_avg_parts(processed, centers)

In [12]:
x = processed.x
cold = get_data_part(processed.averaged, 'cold')
hot = get_data_part(processed.averaged, 'hot')
entropy = processed.average_entropy_signal


In [13]:
cold_fit = calculate_fit(x, cold, params=params, func=i_sense)
hot_fit = calculate_fit(x, hot, params=params, func=i_sense)

entropy_params = lm.Parameters()
entropy_params.add_many(
    ('mid', 0, True, None, None, None, None),
    ('theta', 10, True, 0, 500, None, None),
    ('const', 0, False, None, None, None, None),
    ('dS', 0, True, -5, 5, None, None),
    ('dT', 5, True, -10, 50, None, None)
)
entropy_fit = calculate_fit(x, entropy, params=entropy_params, func=entropy_weak)

amplitude = cold_fit.params['amp'].value
delta_T = hot_fit.params['theta'].value - cold_fit.params['theta'].value
delta_x = np.mean(np.diff(x))
scaling = delta_x/amplitude/delta_T
integrated = np.nancumsum(entropy, axis=-1) * scaling

In [14]:
# TODOs: 
# Neet to integrate entropy signal (using amplitude which is easy, and the dT reported in paper which is 25.9mK == 0.98mV (maybe it is just from fitting hot and cold parts of averaged data))

## Quick plot before exporting data to Igor for final Figure

In [21]:
fig = go.Figure()
fig.update_layout(title='Hot/Cold transition')
fig.add_trace(go.Scatter(x=x, y=cold, mode='markers', marker=dict(color='blue', symbol='circle-open')))
fig.add_trace(go.Scatter(x=x, y=cold_fit.eval(x=x), mode='lines', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=x, y=hot, mode='markers', marker=dict(color='red', symbol='circle-open')))
fig.add_trace(go.Scatter(x=x, y=hot_fit.eval(x=x), mode='lines', line=dict(color='red')))
fig

In [22]:
fig = go.Figure()
fig.update_layout(title='Entropy signal')
fig.add_trace(go.Scatter(x=x, y=entropy, mode='markers', marker=dict(symbol='circle-open')))
fig.add_trace(go.Scatter(x=x, y=entropy_fit.eval(x=x), mode='lines'))
fig

In [28]:
fig = go.Figure()
fig.update_layout(title='Integrated Entropy')
fig.add_trace(go.Scatter(x=x, y=integrated, mode='markers', marker=dict(symbol='circle-open')))
fig.add_shape(dict(type='line', yref='y', y0=np.log(2), y1=np.log(2), xref='paper', x0=0, x1=1,
                  line=dict(color='black', dash='dash')))
fig.add_shape(dict(type='line', yref='y', y0=np.log(3), y1=np.log(3), xref='paper', x0=0, x1=1,
                  line=dict(color='black', dash='dash')))
fig

## Save to .itx file for final Figure in Igor

In [16]:
# save_to_igor_itx(f"exports/WCGW_SWData_{name}.itx", [x]*4, data, [f'WCGW_SWData_{name}_{label}' for label in ['v0_0', 'vP', 'v0_1', 'vP']])